# Variational autoencoder
This notebook will investiga

In [1]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from variational_autoencoder import VariationalAutoencoder
from autoencoder import Autoencoder
from cont_bern_loss import cont_bern_loss

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from latent_plane_mosaic import LatentPlaneMosaic
from latent_interpolation_mosaic import LatentInterpolationMosaic
from sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [2]:
# Global settings
model_path_0 = "models\\vae_100_mse"
model_path_1 = "models\\vae_100"
model_path_2 = "models\\vae_100_cb"
model_path_3 = "models\\vae_1000"
num_samples = 10000
title_0 = "AE"
title_0 = "AE"


np.set_printoptions(precision=0)

## Preprocess data


In [3]:
(mnist_digits, mnist_labels), (test_digits, test_labels) = keras.datasets.mnist.load_data()
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
test_digits = np.expand_dims(test_digits, -1).astype("float32") / 255
input_shape = mnist_digits.shape[1:]
num_digits = len(mnist_labels)
sample_index = rnd.randint(0, num_digits, size = (num_samples,))
mnist_digits = mnist_digits[sample_index]
mnist_labels = mnist_labels[sample_index]

## Load Variational autoencoder

In [4]:
# Variational autoencoder
model_0 = keras.models.load_model(model_path_0, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder})

model_1 = keras.models.load_model(model_path_1, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                         "cont_bern_loss":  cont_bern_loss})

model_2 = keras.models.load_model(model_path_2, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                         "cont_bern_loss":  cont_bern_loss})

model_3 = keras.models.load_model(model_path_3, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder,
                                         "cont_bern_loss":  cont_bern_loss})


### Print setup details

In [6]:
print("Input shape: ", input_shape)
print("Latent dim: ",  model_0.latent_dim)
print("Num parameters: ", model_0.count_params(),
     model_1.count_params(),
     model_2.count_params(),
     model_3.count_params())
latent_dim = model_0.latent_dim
mnist_digits.shape

Input shape:  (28, 28, 1)
Latent dim:  10
Num parameters:  368407 368407 368407 368407


(10000, 28, 28, 1)

## Reconstruct and dimension reductions

In [7]:
# Reconstructions
#mnist_encoded_0 = model_0.encoder(mnist_digits).numpy() 
mnist_distribution_0  = model_1.encoder(mnist_digits).numpy() 
mnist_encoded_0      = mnist_distribution_0[:,0,:]
mnist_log_variance_0  = mnist_distribution_0[:,1,:]

#mnist_encoded_1      = model_1.encoder(mnist_digits).numpy() 
mnist_distribution_1  = model_1.encoder(mnist_digits).numpy() 
mnist_encoded_1      = mnist_distribution_1[:,0,:]
mnist_log_variance_1  = mnist_distribution_1[:,1,:]

#mnist_encoded_2 = model_2.encoder(mnist_digits).numpy() 
mnist_distribution_2 = model_2.encoder(mnist_digits).numpy() 
mnist_encoded_2      = mnist_distribution_2[:,0,:]
mnist_log_variance_2  = mnist_distribution_2[:,1,:]

mnist_distribution_3 = model_3.encoder(mnist_digits).numpy() 
mnist_encoded_3      = mnist_distribution_3[:,0,:]
mnist_log_variance_3 = mnist_distribution_3[:,1,:]

In [8]:
# t-SNE
mnist_tsne2d_0 = TSNE(n_components = 2).fit_transform(mnist_encoded_0)
mnist_tsne2d_1 = TSNE(n_components = 2).fit_transform(mnist_encoded_1)
mnist_tsne2d_2 = TSNE(n_components = 2).fit_transform(mnist_encoded_2)
mnist_tsne2d_3 = TSNE(n_components = 2).fit_transform(mnist_encoded_3)

In [9]:
#pca
pca_0          = PCA().fit(mnist_encoded_0)
mnist_pca_0    = pca_0.transform(mnist_encoded_0)
latent_cov_0   = np.cov(mnist_pca_0.T)
latent_std_0   = np.sqrt(pca_0.explained_variance_)
latent_axes_0  = pca_0.components_
latent_mean_0  = pca_0.mean_

pca_1          = PCA().fit(mnist_encoded_1)
mnist_pca_1    = pca_1.transform(mnist_encoded_1)
latent_cov_1   = np.cov(mnist_pca_1.T)
latent_std_1   = np.sqrt(pca_1.explained_variance_)
latent_axes_1  = pca_1.components_
latent_mean_1  = pca_1.mean_

pca_2          = PCA().fit(mnist_encoded_2)
mnist_pca_2    = pca_2.transform(mnist_encoded_2)
latent_cov_2   = np.cov(mnist_pca_2.T)
latent_std_2   = np.sqrt(pca_2.explained_variance_)
latent_axes_2  = pca_2.components_
latent_mean_2  = pca_2.mean_

pca_3          = PCA().fit(mnist_encoded_3)
mnist_pca_3    = pca_3.transform(mnist_encoded_3)
latent_cov_3   = np.cov(mnist_pca_3.T)
latent_std_3   = np.sqrt(pca_3.explained_variance_)
latent_axes_3  = pca_3.components_
latent_mean_3  = pca_3.mean_


## Investigation

### Show reconstructions

In [10]:

plt.rc('font', family ='Courier New')  


In [11]:
num_col = 4
num_row = 3
num_img = num_row*num_col
rec_index = rnd.randint(num_samples,size =(num_row*num_col,) )
digits = test_digits
digits = digits[rec_index]

In [12]:
model = model_1
fig, ax = plt.subplots()

reconstructions = model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        pair = np.concatenate((digits[i], reconstructions[i]), axis = 1)
        if row == 0: 
            ver_img = pair
        else:
            ver_img = np.concatenate((ver_img,
                                 pair), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)
title = "Reconstruction with autoencoder\non training digits"
aximg = ax.imshow(img,cmap = 'bone', vmin = 0, vmax = 1)
ax.set_title(title,fontsize = 30)
ax.axis("off")
fig.colorbar(aximg)

In [13]:
num_dig = 14
rec_index = rnd.randint(num_samples,size =(num_dig,) )
digits = mnist_digits
digits = digits[rec_index]
fig, ax = plt.subplots()
title = "Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)"
rec0 = model_0(digits)
rec1 = model_1(digits)
rec2 = model_2(digits)
rec3 = model_3(digits)

i = 0
for i in range(num_dig): 
    rec_im = np.concatenate((digits[i],
                         rec0[i],
                         rec1[i],
                         rec2[i],
                         rec3[i]),
                         axis = 0)
    if i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis=1)
    i = i+1
    
ax.imshow(im)
ax.set_title(title)

Text(0.5, 1.0, 'Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)')

In [283]:
rec2[1]


<tf.Tensor: shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
     

### Latent

In [235]:

indeces = rnd.randint(num_samples,size = (3,))
                      
ul = mnist_digits[indeces[0]]
ur = mnist_digits[indeces[1]]
dl = mnist_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)


In [246]:
model = model_3
title = "Latent space using autoencoder"
mosaic = LatentInterpolationMosaic(
                          model.encode,
                          model.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic




"""
fig, axs = plt.subplots(1,2)
axs[0].imshow(corner_image,vmin=0, vmax=1)
axs[1].imshow(mosaic,vmin=0, vmax=1)
axs[0].set_title("Original images beeing procesed")
axs[1].set_title(title)

"""
fig, ax = plt.subplots()
ax.imshow(mosaic,vmin=0, vmax=1)


### Scatter

In [247]:
scatter = mnist_tsne2d_0
SampleScatterGUI(scatter, mnist_labels, mnist_digits)


In [258]:
scatter = mnist_tsne2d_3
unique = np.unique(mnist_labels)
fig,ax = plt.subplots()
for lab in unique:
    group = scatter[mnist_labels == lab]
    ax.scatter(group[:,0], group[:,1], label = str(lab))
ax.legend()

In [251]:
group.shape

(982, 2)

### Spread

In [259]:
fig, ax = plt.subplots(1,1)
ax.plot(latent_std_0, 'o--', label = "AE")
ax.plot(latent_std_1, 'o--', label = "VAE10")
ax.plot(latent_std_2, 'o--', label = "VAE100")
ax.plot(latent_std_3, 'o--', label = "VAE1000")
ax.set_yscale("log")
ax.set_xlabel("Principal axis index in decreasing order")
ax.set_ylabel("Principal variance for the corresponding axis")
ax.set_title("Principal variance of Discrete network")
ax.legend()

In [260]:
c = [np.max(np.abs(latent_cov_0-np.diag(np.diag(latent_cov_0)))),
     np.max(np.abs(latent_cov_1-np.diag(np.diag(latent_cov_1)))),
     np.max(np.abs(latent_cov_2-np.diag(np.diag(latent_cov_2)))),
     np.max(np.abs(latent_cov_3-np.diag(np.diag(latent_cov_3))))]
print("Max non-diagonal covariance ", c)

Max non-diagonal covariance  [3.0864976950513004e-06, 1.887894320384782e-07, 5.131943803213354e-07, 1.398718249464472e-06]


Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [261]:
axes_index = [0,-1]
scaling_factors = np.array([5,5])

In [198]:
model = model_0
latent_vectors = latent_axes_0[axes_index]
latent_origin = latent_mean_0

mosaic = LatentPlaneMosaic(model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

fig,ax = plt.subplots(1,1)
ax.imshow(mosaic)

## Spred 

In [262]:
nbins = 200

In [263]:

dist_0 = np.linalg.norm(mnist_encoded_0-latent_mean_0, axis = 1)
val, bin_edges = np.histogram(dist_0, bins = nbins)
density_x_0 = bin_edges[0:-1]+np.diff(bin_edges)/2
density_0 = val/(density_x_0)**latent_dim

dist_1 = np.linalg.norm(mnist_encoded_1, axis = 1)
std_1 = np.sqrt(np.exp(mnist_log_variance_1.flatten()))
val, bin_edges = np.histogram(dist_1, bins = nbins)
density_x_1 = bin_edges[0:-1]+np.diff(bin_edges)/2
density_1 = val/(density_x_1)**latent_dim

dist_2 = np.linalg.norm(mnist_encoded_2, axis = 1)
std_2 = np.sqrt(np.exp(mnist_log_variance_2.flatten()))
val, bin_edges = np.histogram(dist_2, bins = nbins)
density_x_2 = bin_edges[0:-1]+np.diff(bin_edges)/2
density_2 = val/(density_x_2)**latent_dim

dist_3 = np.linalg.norm(mnist_encoded_3, axis = 1)
std_3 = np.sqrt(np.exp(mnist_log_variance_3.flatten()))
val, bin_edges = np.histogram(dist_3, bins = nbins)
density_x_3 = bin_edges[0:-1]+np.diff(bin_edges)/2
density_3 = val/(density_x_3)**latent_dim

In [264]:

fig, axs = plt.subplots(3,4)
axs[0,0].hist(dist_0, nbins, density = False)
axs[2,0].plot(density_x_0, density_0)


axs[0,1].hist(dist_1, nbins, density = False)
axs[1,1].hist(std_1, nbins,range = (0,1.1), density = True)
axs[2,1].plot(density_x_1, density_1)

axs[0,2].hist(dist_2, nbins, density = False)
axs[1,2].hist(std_2, nbins,range = (0,1.1), density = False)
axs[2,2].plot(density_x_2, density_2)

axs[0,3].hist(dist_3, nbins, density = False)
axs[1,3].hist(std_3, nbins,range = (0,1.1), density = False)
axs[2,3].plot(density_x_3, density_3)

## Generative power

In [265]:
mean = np.zeros(latent_dim)
mean = latent_mean_0
cov  = 1*np.eye(latent_dim)
num_col = 15
num_row = 10
num_img = num_row*num_col
latent_points = rnd.multivariate_normal(mean, cov, size = (num_img))



In [218]:
model = model_0
title = "Reconstruction with autoencoder"
fig, ax = plt.subplots()
reconstructions = model.decoder(latent_points)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        if row == 0: 
            ver_img = reconstructions[i]
        else:
            ver_img = np.concatenate((ver_img,
                                 reconstructions[i]), axis = 0)
        i = i + 1
    if col == 0:
        img = ver_img
    else: 
        img = np.concatenate((img,
                              ver_img), axis = 1)
ax.imshow(img)
ax.set_title(title)


Text(0.5, 1.0, 'Reconstruction with autoencoder')

In [266]:
title = "Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)"
fig, axs = plt.subplots(num_row, num_col)
rec0 = model_0.decoder(latent_points)
rec1 = model_1.decoder(latent_points)
rec2 = model_2.decoder(latent_points)
rec3 = model_3.decoder(latent_points)

i = 0
for col in range(num_col):
    for row in range(num_row): 
        ax = axs[row, col]
        im = np.concatenate((rec0[i],
                             rec1[i],
                             rec2[i],
                             rec3[i]),
                             axis = 1)
        ax.imshow(im)
        ax.set_title(title)
        i = i +1

In [274]:
mean = np.zeros(latent_dim)
cov  = 1*np.eye(latent_dim)
num_dig = 14
latent_points = rnd.multivariate_normal(mean, cov, size = (num_dig))

fig, ax = plt.subplots()
title = "Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)"
rec0 = model_0.decoder(latent_points)
rec1 = model_1.decoder(latent_points)
rec2 = model_2.decoder(latent_points)
rec3 = model_3.decoder(latent_points)


i = 0
for i in range(num_dig): 
    rec_im = np.concatenate((rec0[i],
                         rec1[i],
                         rec2[i],
                         rec3[i]),
                         axis = 0)
    if i == 0:
        im = rec_im
    else:
        im = np.concatenate((im, rec_im), axis=1)
    i = i+1
    
ax.imshow(im)
ax.set_title(title)

Text(0.5, 1.0, 'Progression of models (Org, AE, AE_cb, VAE100,VAE100_cb)')